## GAT

In [1]:
import sys
 
sys.path.insert(0, "../")

In [2]:
from dataAnalysis.DataAnalysis import DataAnalysis
import pandas as pd
import cudf
import torch

data = pd.read_csv(r"../extdata/sbcdata.csv", header=0)
data_analysis = DataAnalysis(data)

/home/dwalke/git/sbc/graph_learning/../dataAnalysis/data/Filter.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['Label'] = self.data['Diagnosis']


Training: 
Assessable data are 528101 cases and 1015074 CBCs
Control data are 527038 cases and 1013548 CBCs
Sepsis data are 1488 cases and 1526 CBCs
$$$$$$$$$$$$$$$$$$$$
Testing: 


/home/dwalke/git/sbc/graph_learning/../dataAnalysis/data/Filter.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['Label'] = self.data['Diagnosis']


Controls: 365794, Sepsis: 490
Assessable data are 180494 cases and 366284 CBCs
Control data are 180157 cases and 365794 CBCs
Sepsis data are 472 cases and 490 CBCs


/home/dwalke/git/sbc/graph_learning/../dataAnalysis/data/Filter.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['Label'] = self.data['Diagnosis']


Controls: 437629, Sepsis: 448
Assessable data are 157922 cases and 438077 CBCs
Control data are 180157 cases and 437629 CBCs
Sepsis data are 438 cases and 448 CBCs


In [3]:
data = pd.concat((data_analysis.get_training_data(), data_analysis.get_testing_data()))

In [4]:
max_Id = data["Id"].unique().max()

In [5]:
gw_data = data_analysis.get_gw_testing_data().copy(deep=True)
gw_data = gw_data.assign(Id=lambda x: x.Id + max_Id)

In [6]:
data = pd.concat((data, gw_data))
data = cudf.from_pandas(data)

In [7]:
data = data.sort_values("Id")

In [8]:
data = data.reset_index(drop=True)

In [9]:
data.pop("index")

0          665587
1          665588
2          665589
3          665590
4          665591
            ...  
1819430    665583
1819431    665581
1819432    665582
1819433    665585
1819434    665586
Name: index, Length: 1819435, dtype: int64

In [10]:
data = data.sort_values(["Age", "Sex"]).reset_index(drop=True)
data = data.to_pandas()

In [11]:
data

,Id,Age,Sex,Diagnosis,Center,Set,Sender,Episode,Time,TargetIcu,SecToIcu,CRP,HGB,MCV,PCT,PLT,RBC,WBC,Label
0,243,18,M,Control,Leipzig,Training,AMB,1,0.0,None,NaN,NaN,9.6,81.3,NaN,301.0,5.72,6.7,Control
1,441,18,M,Control,Leipzig,Training,ED,1,900.0,None,NaN,66.26,5.0,57.4,NaN,366.0,4.46,10.2,Control
2,483,18,M,Control,Leipzig,Training,AMB,1,611760.0,None,NaN,NaN,10.1,84.1,NaN,254.0,5.55,7.4,Control
3,781,18,M,Control,Leipzig,Training,GEN,1,0.0,None,NaN,1.21,9.1,81.1,NaN,313.0,5.35,6.7,Control
4,1444,18,M,Control,Leipzig,Training,AMB,1,0.0,None,NaN,NaN,9.2,82.3,NaN,205.0,5.38,4.5,Control
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1819430,188659,106,W,Control,Leipzig,Training,ED,1,660.0,MICU,4740.0,20.70,7.3,93.7,NaN,313.0,3.65,9.0,Control
1819431,261335,106,W,Control,Leipzig,Training,ED,1,840.0,None,NaN,1.14,7.6,95.3,NaN,281.0,3.84,8.9,Control
1819432,645719,106,W,Control,Leipzig,Validation,ED,1,240.0,None,NaN,3.04,3.7,60.8,NaN,210.0,3.57,8.0,Control
1819433,645719,106,W,Control,Leipzig,Validation,GEN,1,51300.0,None,NaN,9.16,4.2,62.6,NaN,201.0,3.85,6.9,Control


In [12]:
unique_sex_age = data[["Age", "Sex"]].nunique()
unique_sex_age

Age    89
Sex     2
dtype: int64

In [13]:
import numpy as np
np.sum(np.random.uniform(0,1, (10000, 1)) >.8, axis=0)
#np.select([True, True, False], np.array([1,2,3]))

array([1947])

## Creating edge index

In [14]:
# import numpy as np
# import time
# from tqdm.notebook import tqdm

# source_edge_index = np.array([], dtype= np.int32)
# target_edge_index = np.array([], dtype= np.int32)

# start = time.time()
# for Id, group in tqdm(data.groupby(["Age", "Sex"])):
#     indices = group.index
#     indices = np.expand_dims(indices.values, axis = 1)
#     target = np.repeat(indices, indices.shape[0], axis = 1)
#     source = target.transpose().flatten()
#     target = target.flatten()
    
#     mask = np.random.uniform(0,1, (source.shape[0])) >.95

#     source = source[mask]
#     target = target[mask]
    

#     source_edge_index = np.concatenate((source_edge_index, source))
#     target_edge_index = np.concatenate((target_edge_index, target))
    

# print(time.time() - start )
# print(source_edge_index.shape)
# print(target_edge_index.shape)

# edge_index = np.asarray([np.asarray(source_edge_index), np.asarray(target_edge_index)])
# edge_index = torch.tensor(edge_index)
# pd.DataFrame(edge_index.numpy().transpose()).to_csv("edge_index_sorted_age_sex_w_gw.csv", index=False)

In [16]:
import torch
edge_index = torch.tensor(pd.read_csv("edge_index_sorted_age_sex_w_gw.csv", header=None, skiprows=1).values.transpose(), dtype=torch.long)

In [17]:
edge_index.shape

torch.Size([2, 1342601724])

In [18]:
from dataAnalysis.Constants import SEX_CATEGORY_COLUMN_NAME, SEX_COLUMN_NAME, FEATURES
data[SEX_CATEGORY_COLUMN_NAME] = data.loc[:, SEX_COLUMN_NAME] =="W"

In [19]:
data[SEX_CATEGORY_COLUMN_NAME] = data[SEX_CATEGORY_COLUMN_NAME].astype("int8")
data["Label"] = data["Label"] == "Sepsis"
data["Label"] = data["Label"].astype("int8")

In [20]:
X_features = torch.tensor(data[FEATURES].values).type(torch.float32)
y = torch.tensor(data["Label"].values).type(torch.long)

In [21]:
from dataAnalysis.FeatureImportance import normalize

X_features = normalize(X_features)

In [22]:
X_new_features = X_features

In [23]:
X_new_features.shape

torch.Size([1819435, 7])

In [24]:
import numpy as np
def ratio_bool_switch(tensor, ratio = 0.8):
    random = np.random.uniform(0, 1 ,tensor.shape[0])
    val_ratio_mask = (random >= 0.8)
    train_ratio_mask = (random < 0.8)
    val_mask = np.logical_and(tensor.tolist(), val_ratio_mask.tolist())
    train_mask = np.logical_and(tensor.tolist(), train_ratio_mask.tolist())
    return torch.from_numpy(train_mask).type(torch.bool), torch.from_numpy(val_mask).type(torch.bool)

In [25]:
from torch_geometric.data import Data

train_mask_ser = data["Set"] != "Validation"
train_mask, val_mask = ratio_bool_switch(train_mask_ser.values)
test_mask = torch.from_numpy(np.logical_and((data["Set"] == "Validation").values, (data["Center"] == "Leipzig").values)).type(torch.bool)
test_gw_mask = torch.from_numpy(np.logical_and((data["Set"] == "Validation").values, (data["Center"] == "Greifswald").values)).type(torch.bool)
graph = Data(x=X_new_features, train_mask = train_mask, test_mask=test_mask, val_mask=val_mask, y= y, edge_index=edge_index,
             test_gw_mask = test_gw_mask)

In [26]:
test_gw_mask.sum() + test_mask.sum()+val_mask.sum()+train_mask.sum()

tensor(1819435)

In [27]:
graph

Data(x=[1819435, 7], edge_index=[2, 1342601724], y=[1819435], train_mask=[1819435], test_mask=[1819435], val_mask=[1819435], test_gw_mask=[1819435])

In [28]:
graph = graph.cpu()

In [ ]:
from torch_geometric.loader import NeighborLoader

kwargs = {
    "num_neighbors":[50] * 2,
    "batch_size":10_000
}
loader = NeighborLoader(
    graph,
    input_nodes=graph.train_mask,
    **kwargs
)
val_loader = NeighborLoader(
    graph,
    input_nodes=graph.val_mask,
    **kwargs
)

In [ ]:
device = torch.device("cuda:1") if torch.cuda.is_available() else torch.device("cpu")

# graph = graph.to(device)
WEIGHT = torch.tensor([664*4/5])#
WEIGHT = WEIGHT.to(device)

print("Data shifted to the device " + str(device))

In [ ]:
import torch.nn.functional as F
from torch_geometric.nn import GATConv, GCNConv,GATv2Conv, GINConv, global_add_pool
from torch.nn import Linear
import torch
from dataAnalysis.Constants import FEATURES
from torch.nn import Linear, ReLU, Sequential
from torch.nn import BatchNorm1d as BatchNorm

class GraphNeuralNetwork(torch.nn.Module):
    def __init__(self, hidden_dim = 128, out_channels = 1):
        super(GraphNeuralNetwork, self).__init__()
        input_dim = len(FEATURES)      
        
        HEADS = 2
        
        conv_1= GATConv(input_dim, hidden_dim,heads=HEADS, add_self_loops = False)
        conv_end = GATConv((-1,-1), out_channels,add_self_loops = False)
        
        self.conv_1 = conv_1
        self.conv_end = conv_end
        

    def forward(self, graph):
        x, edge_index = graph.x, graph.edge_index
        x = x.type(torch.float)
        x = self.conv_1(x, edge_index)
        x = F.normalize(x, p=2., dim=-1)
        x = torch.relu(x)
        x = self.conv_end(x, edge_index)
        return x
            
    def predict_proba(self, graph, mask):
        with torch.inference_mode():
            self.eval()
            logits = self.forward(graph)
            scores = torch.sigmoid(torch.squeeze(logits[mask]))
            scores = torch.unsqueeze(scores, 0)
            proba_predict = torch.concat((1- scores, scores), dim = 0)
            return torch.transpose(proba_predict, 0, 1)
            
    def predict(self, graph, mask):
        return torch.round(self.predict_proba(graph, mask)[:, 1])

In [ ]:
import torch 
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
class ModelWrapper():
    def __init__(self, graph):
        self.LEARNING_RATE = 3e-3#3e-4
        self.MAX_EPOCHS = 10#1000 #10000 #40000

        self.model = GraphNeuralNetwork(hidden_dim = 128, out_channels=1) 
        self.model = self.model.to(device)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.LEARNING_RATE,betas=(0.9, 0.999), eps=1e-08)
        self.graph = graph
        
        self.last_loss = 0
        self.increased_loss = 0
        self.BREAKING_THRESHOLD = 5 
        self.val_loss = []
        self.train_loss = []
        self.epochs = self.MAX_EPOCHS
    
    def validate(self):
        with torch.inference_mode():
            self.model.eval()
            acc_loss = 0
            batch_size = 0
            for batch, graph in enumerate(val_loader):
                graph = graph.to(device)
                out = self.model(graph)
                loss = F.binary_cross_entropy_with_logits(torch.squeeze(out), graph.y.type(torch.float32),
                                                          pos_weight=WEIGHT)
                acc_loss += loss.item()
                batch_size += 1
            avg_loss = acc_loss / batch_size
            self.val_loss.append(avg_loss)
            if avg_loss > self.last_loss:
                self.increased_loss += 1
            else:
                self.increased_loss = 0
            self.last_loss = avg_loss

    def train(self):
        for epoch in tqdm(range(self.MAX_EPOCHS)):
            acc_loss = 0
            batch_size = 0
            for batch, graph in enumerate(loader):
                graph = graph.to(device)
                self.model.train()
                self.optimizer.zero_grad()
                out = self.model(graph)
                loss = F.binary_cross_entropy_with_logits(torch.squeeze(out), graph.y.type(torch.float32),
                                                          pos_weight=WEIGHT)
                acc_loss += loss.item()
                batch_size += 1
                loss.backward()
                self.optimizer.step()
            self.train_loss.append(acc_loss/batch_size)
            self.validate() 

            if self.increased_loss >= self.BREAKING_THRESHOLD:
                self.epochs = epoch + 1
                print(f"Breaked at {str(epoch)}")
                break
            
    def get_model(self):
        return self.model    
    
    def plot_loss(self):
        plt.plot(range(len(self.train_loss)), self.train_loss, 'g', label='Training loss')
        plt.plot(range(len(self.val_loss)), self.val_loss, 'y', label='Validation loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
        plt.show()

In [ ]:
model_wrapper = ModelWrapper(graph)
model_wrapper.train()

In [ ]:
import matplotlib.pyplot as plt
model_wrapper.plot_loss()

In [35]:
model = model_wrapper.get_model()
graph = graph.cpu()
model = model.cpu()

In [37]:
test_loader = NeighborLoader(
    graph,
    input_nodes=graph.test_mask,
    **kwargs
)
test_gw_loader = NeighborLoader(
    graph,
    input_nodes=graph.test_gw_mask,
    **kwargs
)

In [50]:
from sklearn.metrics import roc_auc_score
def evaluate(loader, mask):
    y_pred_proba_all = torch.tensor([])
    y_true_all = torch.tensor([])
    for batch, test_graph in enumerate(loader):
        with torch.inference_mode():
            model.eval()
            out = model(test_graph)
            scores = torch.sigmoid(torch.squeeze(out[test_graph[mask]]))
#             scores = torch.unsqueeze(scores, 0)
            y_pred_proba_all = torch.concat((y_pred_proba_all, scores), dim = 0)
            y_true_all = torch.concat((y_true_all, test_graph.y[test_graph[mask]]), dim = 0)
            print(y_pred_proba_all)
            print(roc_auc_score(y_true_all, y_pred_proba_all))
    return y_pred_proba_all, y_true_all

y_pred_proba, y_true = evaluate(test_loader, "test_mask")
auroc = roc_auc_score(y_true, y_pred_proba)
            

tensor([0.1941, 0.1939, 0.1964,  ..., 0.4026, 0.4026, 0.4026])
0.7915429906087295
tensor([0.1941, 0.1939, 0.1964,  ..., 0.4026, 0.4026, 0.4026])
0.7489165416633466
tensor([0.1941, 0.1939, 0.1964,  ..., 0.4026, 0.4026, 0.4026])
0.6788290595554417
tensor([0.1941, 0.1939, 0.1964,  ..., 0.4026, 0.4026, 0.4026])
0.6551692553726347
tensor([0.1941, 0.1939, 0.1964,  ..., 0.4026, 0.4026, 0.4026])
0.6540059601177484
tensor([0.1941, 0.1939, 0.1964,  ..., 0.4026, 0.4026, 0.4026])
0.6890531467621455
tensor([0.1941, 0.1939, 0.1964,  ..., 0.4026, 0.4026, 0.4026])
0.6985154258148536
tensor([0.1941, 0.1939, 0.1964,  ..., 0.4026, 0.4026, 0.4026])
0.7137944319264533
tensor([0.1941, 0.1939, 0.1964,  ..., 0.4026, 0.4026, 0.4026])
0.7026159873412554
tensor([0.1941, 0.1939, 0.1964,  ..., 0.4026, 0.4026, 0.4026])
0.6980931871433411
tensor([0.1941, 0.1939, 0.1964,  ..., 0.4026, 0.4026, 0.4026])
0.688636379753457
tensor([0.1941, 0.1939, 0.1964,  ..., 0.4026, 0.4026, 0.4026])
0.6879329436713258
tensor([0.1941, 0

In [47]:
auroc

0.6674597093838381

In [36]:
from dataAnalysis.Metrics import Evaluation

y_dict = Evaluation.create_y_dict(model.predict(graph, test_mask), model.predict_proba(graph, test_mask) , graph.y[test_mask])
y_dict_gw = Evaluation.create_y_dict(model.predict(graph, test_gw_mask), model.predict_proba(graph, test_gw_mask) , graph.y[test_gw_mask])

RuntimeError: [enforce fail at alloc_cpu.cpp:75] err == 0. DefaultCPUAllocator: can't allocate memory: you tried to allocate 3437060413440 bytes. Error code 12 (Cannot allocate memory)

In [ ]:
from dataAnalysis.Metrics import Evaluation

Evaluation.plot_confusion_matrix_from_pred(model.predict(graph, test_mask), graph.y[test_mask])
Evaluation.plot_confusion_matrix_from_pred(model.predict(graph, test_gw_mask), graph.y[test_gw_mask])
Evaluation.get_df_metrics_from_pred(y_dict, y_dict_gw)

In [ ]:
from IPython.display import Audio
sound_file = './finish_sound.mp3'
Audio(sound_file, autoplay=True)

In [56]:
##TODO:
# - change weight
# - gradients as features
# - different learning rate and number of layers
# - general architecture- less neurons but linear layer before and after convs? 
#       -adding hidden layers with each others for more stable training (guaranteed only when hidden dim is euqal betwene layers)
# - GATv2
# - MIMIC und Greifswald
# - more features

In [ ]:
import time

for i in range(10000):
    time.sleep(60)
    print("Sleeping")

Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
S